# Building Rental Property Dataset Using Web Scraping

#### Team Members:<br>
Madhu Bandru - MB4236<br>
Varun Kumar B - VB475<br>
Karthikreddy Kuna - KK3375<br>
Pradeep Kumar Kankipati - PK593

In [3]:
#importing required libraries
import requests, re, pandas
from bs4 import BeautifulSoup
from collections import defaultdict

In [4]:
#extracting top market cities and their states

base_url = "https://www.apartments.com/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
response = requests.get(base_url, headers=headers)
soup1 = BeautifulSoup(response.text,"html.parser")

city_url = soup1.find_all("div",{"class":"clearfix"})
list_city=[]
for i in city_url:
     data=i.find_all("li")
for j in data:
    if re.search(r'Pet',j.find("a")["title"]):
        break
    else:
        list_city.append(j.find("a")["href"])
list_city

['/atlanta-ga/',
 '/austin-tx/',
 '/baltimore-md/',
 '/boston-ma/',
 '/chicago-il/',
 '/cincinnati-oh/',
 '/dallas-tx/',
 '/denver-co/',
 '/detroit-mi/',
 '/houston-tx/',
 '/irvine-ca/',
 '/los-angeles-ca/',
 '/miami-fl/',
 '/nashville-tn/',
 '/nj/',
 '/new-york-ny/',
 '/orlando-fl/',
 '/philadelphia-pa/',
 '/phoenix-az/',
 '/portland-or/',
 '/queens-ny/',
 '/san-antonio-tx/',
 '/san-diego-ca/',
 '/san-francisco-ca/',
 '/seattle-wa/',
 '/tampa-fl/',
 '/washington-dc/']

In [5]:
#extracting individual property URL's for a city


def get_url_list(each_city):
    url = "https://www.apartments.com"+each_city
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    response = requests.get(url, headers=headers)
    c=response.text
    soup = BeautifulSoup(c,"html.parser")


    ####get the last page number
    num_list = ''
    page_num = soup.find_all("nav", {"class":"paging"})

    for tags in page_num:
        if tags.find('a'):
            num_list += ((tags.text).strip())
    page_num_list = num_list.replace("\n","").replace(".","")

    last_page_num = page_num_list.split(' ')[len(page_num_list.split(' ')) - 1]

    if last_page_num != '5Next':
        last_page = int(last_page_num)
    else:
        page_num = soup.find_all('p', {'class' : "searchResults"})
        for tags in page_num:
            if tags.find('span'):
                last_page = int(((tags.text).split(' '))[len((tags.text).split(' ')) - 1])

    ####get urls of all properties in all pages
    url_list = []
    current_page = 1
    while current_page < (last_page + 1):

        if current_page == 1:
            url = "https://www.apartments.com"+each_city
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
            response = requests.get(url, headers=headers)
            c=response.text
            soup = BeautifulSoup(c,"html.parser")
        else:
            url = "https://www.apartments.com"+each_city+str(current_page)+"/"
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
            response = requests.get(url, headers=headers)
            c=response.text
            soup = BeautifulSoup(c,"html.parser")

        #appending property URL's to a list.
        placard_header = soup.find_all("li")
        for data in placard_header:
            if data.find('article'):
                data1 = data.find('article')
                url_link = data1.get('data-url')
                if url_link == None:
                    data1 = data.find('a')
                    url_link = data1.get('href')
                    url_list.append(url_link) 
                else:
                    url_list.append(url_link)    
        current_page += 1        
    return url_list

In [6]:
#Provide the city and state(abbrevation) in city_state field.

city_state = '/philadelphia-pa/'
if city_state not in list_city:
    print('Invalid city or state')
else:
    property_url_list = get_url_list(city_state)

property_url_list[:10]

['https://www.apartments.com/fallsview-gardens-philadelphia-pa/xk810sq/',
 'https://www.apartments.com/hanover-north-broad-philadelphia-pa/540bw8m/',
 'https://www.apartments.com/chestnut-hill-village-philadelphia-pa/4sbjxeh/',
 'https://www.apartments.com/the-irvine-philadelphia-pa/61hve9v/',
 'https://www.apartments.com/brookview-apartments-at-elkins-park-elkins-park-pa/ked8x69/',
 'https://www.apartments.com/lynnewood-gardens-elkins-park-pa/7bjgd3y/',
 'https://www.apartments.com/colonial-point-apartments-feasterville-trevose-pa/sd0qgc7/',
 'https://www.apartments.com/tower-place-philadelphia-pa/bex449c/',
 'https://www.apartments.com/rushwood-apartments-philadelphia-pa/g8qsrgz/',
 'https://www.apartments.com/the-piazza-philadelphia-pa/pslb7ts/']

In [ ]:
#extracting required fields to build the data set for each property in the above list.

property_details = defaultdict(dict)
i = 0
address_var_list = []
for new_url in property_url_list:
    i += 1

    new_response = requests.get(new_url, headers=headers)
    new_c=new_response.text
    new_soup = BeautifulSoup(new_c,"html.parser")

    #####Locate and store Property name in dictionary
    property_name = (new_soup.find("h1", {"class":"propertyName"}).text).strip()
    property_details[i]['property name'] = property_name

    #####Locate and store Property adress in dictionary
    data1 = ''
    property_address = new_soup.find_all("h2")
    for tags in property_address:
        if tags.find('span'):
            data1 += ((tags.text).strip())
    data2 = (data1.replace("\r","").replace("  ","").replace(",","")).split('\n')

    ####few properties have street name as property name.
    if len(data2) == 4:
        property_details[i]['property street'] = data2[0]
        property_details[i]['property city'] = data2[1]
        property_details[i]['property state'] = data2[2]
        property_details[i]['property zipcode'] = data2[3]
    else:
        property_details[i]['property street'] = property_name
        property_details[i]['property city'] = data2[0]
        property_details[i]['property state'] = data2[1]
        property_details[i]['property zipcode'] = data2[2]        

    #####Locate and store Property features in dictionary
    feature_data = ''
    property_features = new_soup.find_all("div", {"class":"rentRollupContainer"})
    for tags in property_features:
        if tags.find('span'):
            feature_data += (tags.text).strip()

    feature_data1 = (feature_data.replace("\t","").replace("\n","")).split("\r")
    property_details[i]['Studio'] = 'NA'
    property_details[i]['1 Bedroom'] = 'NA'
    property_details[i]['2 Bedrooms'] = 'NA'
    property_details[i]['3 Bedrooms'] = 'NA'
    for tags in range(0, len(feature_data1), 3):
        if feature_data1[tags] == 'StudioStudio':
            property_details[i]['Studio'] = feature_data1[tags+2].strip()
        elif feature_data1[tags] == '1 Bedroom1 Bed':
            property_details[i]['1 Bedroom'] = (feature_data1[tags+2]).strip()
        elif feature_data1[tags] == '2 Bedrooms2 Beds':
            property_details[i]['2 Bedrooms'] = feature_data1[tags+2].strip()
        elif feature_data1[tags] == '3 Bedrooms3 Beds':
            property_details[i]['3 Bedrooms'] = feature_data1[tags+2].strip()

    ####Latitude and Longitude of property.
    latitude,longitude = new_soup.find_all("meta", property={"place:location:latitude","place:location:longitude"})
    property_details[i]['Latitude'] = latitude["content"]
    property_details[i]['Longitude'] = longitude["content"]    
    
    #####Append URL 
    property_details[i]['URL'] = new_url

property_details

In [9]:
#storing the data in CSV file using pandas.

df = pandas.DataFrame(property_details).T
neworder = ['property name', 'property street','property city','property state','property zipcode','Studio', '1 Bedroom', 
            '2 Bedrooms', '3 Bedrooms', 'Latitude', 'Longitude', 'URL' ]
df=df.reindex(columns=neworder)
df.to_csv("data/rental_data_set.csv", encoding = 'utf-8-sig')

In [1]:
df[10:20]

NameError: name 'df' is not defined